In [2]:
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
ply_path = "D:\sunny\Codes\IIB_project\data\michaelmas\ear.ply"
mesh = o3d.io.read_triangle_mesh(ply_path)

vertex_positions = np.asarray(mesh.vertices)
centroid = np.mean(vertex_positions, axis=0)
# vertex_positions -= centroid
print(centroid)


[40.60311518 44.69366868 61.70447312]


In [4]:

input_txt_path= "D:\\sunny\\Codes\\IIB_project\\data\\lent\\rwnContour.txt"
output_txt_path= "D:\\sunny\\Codes\\IIB_project\\data\\lent\\rwnContour_center.txt"
offset = centroid

with open(input_txt_path, 'r') as fin, open(output_txt_path, 'w') as fout:
    for line in fin:
        line_stripped = line.strip()

        # Check if this line starts with 'comment'
        if not line_stripped.startswith("comment"):
            # If it's not a "comment" line, just copy it over
            fout.write(line)
            continue

        # Split into tokens
        parts = line_stripped.split()

        # If the line has exactly 7 tokens, e.g.:
        # comment X Y Z Nx Ny Nz
        # then we parse them. Otherwise, we just preserve the line as-is.
        # (lines like 'comment CLOSED_LANDCURVE "round window" 27' have more/different tokens)
        if len(parts) == 7:
            # parts[0] == 'comment'
            try:
                x, y, z = map(float, parts[1:4])
                nx, ny, nz = map(float, parts[4:7])
            except ValueError:
                # If we can't parse floats, just copy the line
                fout.write(line)
                continue

            # Subtract offset from position
            x_new = x - offset[0]
            y_new = y - offset[1]
            z_new = z - offset[2]

            # Keep normal as is
            nx_new = nx
            ny_new = ny
            nz_new = nz

            # Rebuild the line in the same format
            new_line = f"comment {x_new:.5f} {y_new:.5f} {z_new:.5f} {nx_new:.5f} {ny_new:.5f} {nz_new:.5f}\n"
            fout.write(new_line)

        else:
            # Not a numeric coordinate+normal line; copy it directly
            fout.write(line)

In [5]:
curve_points = []

with open(input_txt_path, "r") as f:
    for line in f:
        line_stripped = line.strip()
        if not line_stripped.startswith("comment"):
            # e.g. skip lines that don't start with comment
            continue
        
        parts = line_stripped.split()
        # If the line has 7 tokens (comment + 6 floats),
        # then parse them as x,y,z,nx,ny,nz
        if len(parts) == 7:
            try:
                x, y, z = map(float, parts[1:4])
                # nx, ny, nz = map(float, parts[4:7])  # we won't need these for the line geometry
                curve_points.append([x, y, z])
            except ValueError:
                # The line might be 'comment CLOSED_LANDCURVE "..." N', just skip
                continue

curve_points = np.array(curve_points)

# Connect the last point to the first point to close the curve
N = len(curve_points)
lines = [[i, i + 1] for i in range(N - 1)]
lines.append([N - 1, 0])  # close the curve

# Create a LineSet object
line_set = o3d.geometry.LineSet()
line_set.points = o3d.utility.Vector3dVector(curve_points)
line_set.lines = o3d.utility.Vector2iVector(lines)

# Color all lines red (just an example)
colors = [[1.0, 0.0, 0.0] for _ in lines]
line_set.colors = o3d.utility.Vector3dVector(colors)


In [6]:
# Optionally compute vertex normals if they are not already provided
mesh.compute_vertex_normals()

# Visualize the mesh in an interactive Open3D window
o3d.visualization.draw_geometries([mesh, line_set],
                                  window_name="Open3D Mesh Viewer",
                                  width=800,     # optional window width
                                  height=600)    # optional window height

In [10]:
opticCap_path = "D:\\sunny\\Codes\IIB_project\\data\\lent\\OssiclesOnly.ply"
opticCapMesh = o3d.io.read_triangle_mesh(opticCap_path)
opticCapMesh.compute_vertex_normals()
# change all vertex colors into red
opticCapMesh.paint_uniform_color([0, 0, 1])
mesh.paint_uniform_color([0, 1, 0])


o3d.visualization.draw_geometries([mesh, opticCapMesh, line_set],
                                  window_name="Open3D Mesh Viewer",
                                  width=800,     # optional window width
                                  height=600)    # optional window height